In [1]:
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import numpy as np
import genjax
from genjax import GenerativeFunction, ChoiceMap, Selection, trace

# Pretty printing.
console = genjax.pretty(show_locals=True, width=70)

# Reproducibility.
key = jax.random.PRNGKey(314159)

## What is GenJAX?

Here are a few high-level ways to think about GenJAX:

* A probabilistic programming system based on the concepts of [Gen](https://www.gen.dev/).

* A Bayesian model and inference compiler with support for device acceleration (courtesy of JAX).

* A base layer for experiments in model and inference DSL design.

There's already a well-supported implementation of [Gen in Julia](https://github.com/probcomp/Gen.jl). Why is a JAX port interesting?

There are a number of compelling reasons to explore Gen's probabilistic programming paradigm on top of JAX, here are a few:

* JAX's excellent accelerator support - our implementation natively supports several common accelerator idioms - like automatic struct-of-array representations, and the ability to automatically batch model/inference programs onto accelerators.

* JAX's excellent support for compositional higher-order AD allows us to explore higher-order AD in inference design, previously a difficult implementation challenge.

* JAX exposes compositional code transformations to library authors, and, as library authors, we can utilize code transformations to implement state-of-the-art optimizations for models and inference expressed in our system.

Let's truncate the list there for now.

For the JAX literati, one final (hopefully tantalizing) takeaway: <u>by construction, all GenJAX modeling + inference code is JAX traceable</u> - and thus, `vmap`able, `jit`able, etc.

## What is probabilistic programming?

Perhaps you may be coming to this notebook without any prior knowledge in probabilistic programming...

That's okay! Ideally, the ideas in this notebook should be self-contained^[You may miss _why generative functions (see below) are designed the way they are_ on a first read - but you'll still get the punchline if you follow the notebook to the end.].

### A Bayesian viewpoint

Here's one take on what probabilistic programming is all about: programming language design for expressing and solving Bayesian inference problems.

In the [Probabilistic Computing lab at MIT](http://probcomp.csail.mit.edu/), we also consider differentiable programming to be contained within the set of concerns of probabilistic programming.

Probabilistic programming is a broad field, and there are corners which may not be covered by this viewpoint. Mostly, people are interested in Bayes, and how to represent Bayes on computers in nice ways.

### What are we actually computing with?

As you'll see if you continue to read below, the objects which we program with expose a mixture of generative and differentiable interfaces - the interfaces are designed to support common (as well as quite advanced) classes of Bayesian inference algorithms. Gen provides automation for the tricky math which these algorithms sometimes require.

We separate the design of inference (whose implementation uses the interfaces), from the implementation of the interfaces on computational objects. This allows us to build languages of objects which satisfy the interfaces - and allows their interoperability and usage.

In Gen, we call objects which implement the interfaces **generative functions**.

## What is a generative function?

Generative functions are the key concept of Gen's probabilistic programming paradigm. Generative functions are computational objects defined by a set of associated data types and methods. These types and methods describe compositional interfaces useful for inference computations. 

Gen's formal description of generative functions consist of two objects:

* $P(\tau, r; x)$ - a normalized measure over tree-like data (*choice maps*) and untraced randomness^[More on this later. It's safe to say "I have no idea what that is" for now, and expect us to explain later.] $r$, parametrized by arguments $x$.

* $f(\tau; x)$ - a deterministic function from the above measure's sample space to a space of data types.

We can informally think of the sampling semantics of these objects as consisting of two steps:

1. First, sample a  choice map from $P$.
2. Then, compute the return value using $f$.

In many of the generative function interfaces, we won't just be interested in the final sampled return value. We'll also be interested in what happened along the way: we'll record the intermediate and final results of these steps in `Trace` objects - data structures which contain the recordings of values, along with probabilistic metadata like the score of random choices selected along the way.

Here's an example of a GenJAX generative function^[There's not just one generative function class - users can and are encouraged to design new types of generative functions which capture repeated modeling patterns. An excellent example of this modularity in Gen's design is [generative function combinators](/introduction/intro_to_combinators/intro_to_combinators).]. 

This generative function is part of a function-like language - pay close attention to the hierarchical compositionality of generative functions in this language under an abstraction (`genjax.trace`) similar to a function call. 

We'll discuss the addresses (`"sub"` and `"m0"`) a bit later.

In [2]:
@genjax.gen
def g(key, x):
    key, m0 = genjax.trace("m0", genjax.Bernoulli)(key, x)
    return (key, m0)


@genjax.gen
def h(key, x):
    key, m0 = genjax.trace("sub", g)(key, x)
    return (key, m0)


h

BuiltinGenerativeFunction
└── source
    └── <function h>

This generative function holds a Python `Callable` object. For this generative function language, the interface methods (see the list under **Generative function interface** below) which are useful for modeling and inference are given semantics via JAX's tracing and program transformation infrastructure.

Let's examine some of these operations now.

In [3]:
console.inspect(genjax.BuiltinGenerativeFunction, methods=True)

╭─ <class 'genjax.generative_functions.builtin.builtin_gen─╮
│ class BuiltinGenerativeFunction(source: Callable) -> Non │
│                                                          │
│ BuiltinGenerativeFunction(source: Callable)              │
│                                                          │
│         assess = def assess(self, key, chm, args,        │
│                  **kwargs):                              │
│        flatten = def flatten(self):                      │
│ get_trace_type = def get_trace_type(self, key, args,     │
│                  **kwargs):                              │
│     importance = def importance(self, key, chm, args,    │
│                  **kwargs):                              │
│       simulate = def simulate(self, key, args,           │
│                  **kwargs):                              │
│      unflatten = def unflatten(data, xs):                │
│          unzip = def unzip(self, key:                    │
│                  jaxtyping.Integer[Array, '...'], fixed: │
│                  genjax.core.datatypes.ChoiceMap) ->     │
│                  Tuple[jaxtyping.Integer[Array, '...'],  │
│                  Callable[[genjax.core.datatypes.Choice… │
│                  Tuple], float],                         │
│                  Callable[[genjax.core.datatypes.Choice… │
│                  Tuple], Any]]:                          │
│         update = def update(self, key, prev, new, args,  │
│                  **kwargs):                              │
╰──────────────────────────────────────────────────────────╯

This is our first glimpse of the **generative function interface** (GFI), the secret sauce which Gen is based around.

::: {.callout-note}

## JAX interfaces

There's a few methods here which are not part of the GFI, but are worth mentioning because they deal with data interfaces to JAX:

* `flatten` - which allows us to treat generative functions as [Pytree](https://jax.readthedocs.io/en/latest/pytrees.html) implementors.
* `unflatten` - same as above.

These are used to register the implementor type as a `Pytree`, which is roughly a tree-like Python structure which JAX can zip/unzip at runtime API boundaries.

:::

Let's study the `simulate` method first: we'll explore its semantics, and see the types of data it produces.

In [4]:
key, tr = genjax.simulate(h)(key, (0.3,))
tr

BuiltinTrace
├── gen_fn
│   └── BuiltinGenerativeFunction
│       └── source
│           └── <function h>
├── args
│   └── (0.3,)
├── retval
│   └── bool[]
├── choices
│   └── BuiltinChoiceMap
│       └── sub
│           └── BuiltinTrace
│               ├── gen_fn
│               │   └── BuiltinGenerativeFunction
│               │       └── source
│               │           └── <function g>
│               ├── args
│               │   └── (0.3,)
│               ├── retval
│               │   └── bool[]
│               ├── choices
│               │   └── BuiltinChoiceMap
│               │       └── m0
│               │           └── DistributionTrace
│               │               ├── gen_fn
│               │               │   └── _Bernoulli
│               │               ├── args
│               │               │   └── (0.3,)
│               │               ├── value
│               │               │   └── ValueChoiceMap
│               │               │       └── value
│               │               │           └── bool[]
│               │               └── score
│               │                   └── f32[]
│               ├── cache
│               │   └── BuiltinTrie
│               └── score
│                   └── f32[]
├── cache
│   └── BuiltinTrie
└── score
    └── f32[]

If you're familiar with other "trace-based" probabilistic systems - this should look familiar. 

This object instance is a piece of data which has captured information about the execution of the function. Specifically, the subtraces of _other generative function calls_ which occur in `genjax.trace` statements.

It also captures the `score` - the log probability of the normalized measure which the model program represents, evaluated at the random choices which the generative call execution produced. 

If you were paying attention above, the score is $\log P(\tau, r; x)$.

### How is `simulate` implemented for this language?

For this generative function language, we implement `simulate` using a code transformation! Here's the transformed code.

In [5]:
jaxpr = jax.make_jaxpr(genjax.simulate(h))(key, (0.3,))
jaxpr

{ lambda ; a:u32[2] b:f32[]. let
    c:key<fry>[] = random_wrap[impl=fry] a
    d:key<fry>[2] = random_split[count=2] c
    e:u32[2,2] = random_unwrap d
    f:u32[1,2] = slice[limit_indices=(1, 2) start_indices=(0, 0) strides=(1, 1)] e
    g:u32[2] = squeeze[dimensions=(0,)] f
    h:u32[1,2] = slice[limit_indices=(2, 2) start_indices=(1, 0) strides=(1, 1)] e
    i:u32[2] = squeeze[dimensions=(0,)] h
    j:key<fry>[] = random_wrap[impl=fry] i
    k:f32[] = convert_element_type[new_dtype=float32 weak_type=False] b
    l:u32[] = random_bits[bit_width=32 shape=()] j
    m:u32[] = shift_right_logical l 9
    n:u32[] = or m 1065353216
    o:f32[] = bitcast_convert_type[new_dtype=float32] n
    p:f32[] = sub o 1.0
    q:f32[] = sub 1.0 0.0
    r:f32[] = mul p q
    s:f32[] = add r 0.0
    t:f32[] = reshape[dimensions=None new_sizes=()] s
    u:f32[] = max 0.0 t
    v:bool[] = lt u k
    w:f32[] = convert_element_type[new_dtype=float32 weak_type=True] v
    x:f32[] = sub w 0.0
    y:bool[] = ne x 0.0
    z:f32[] = xla_call[
      call_jaxpr={ lambda ; ba:bool[] bb:f32[] bc:f32[]. let
          bd:bool[] = convert_element_type[new_dtype=bool weak_type=False] ba
          be:f32[] = select_n bd bc bb
        in (be,) }
      name=_where
    ] y x 1.0
    bf:f32[] = xla_call[
      call_jaxpr={ lambda ; bg:bool[] bh:f32[] bi:f32[]. let
          bj:bool[] = convert_element_type[new_dtype=bool weak_type=False] bg
          bk:f32[] = select_n bj bi bh
        in (bk,) }
      name=_where
    ] y b 1.0
    bl:f32[] = log bf
    bm:f32[] = mul z bl
    bn:f32[] = xla_call[
      call_jaxpr={ lambda ; bo:bool[] bp:f32[] bq:f32[]. let
          br:bool[] = convert_element_type[new_dtype=bool weak_type=False] bo
          bs:f32[] = select_n br bq bp
        in (bs,) }
      name=_where
    ] y bm 0.0
    bt:f32[] = sub 1.0 x
    bu:f32[] = neg b
    bv:bool[] = ne bt 0.0
    bw:f32[] = xla_call[
      call_jaxpr={ lambda ; bx:bool[] by:f32[] bz:f32[]. let
          ca:bool[] = convert_element_type[new_dtype=bool weak_type=False] bx
          cb:f32[] = select_n ca bz by
        in (cb,) }
      name=_where
    ] bv bt 1.0
    cc:f32[] = xla_call[
      call_jaxpr={ lambda ; cd:bool[] ce:f32[] cf:f32[]. let
          cg:bool[] = convert_element_type[new_dtype=bool weak_type=False] cd
          ch:f32[] = select_n cg cf ce
        in (ch,) }
      name=_where
    ] bv bu 1.0
    ci:f32[] = log1p cc
    cj:f32[] = mul bw ci
    ck:f32[] = xla_call[
      call_jaxpr={ lambda ; cl:bool[] cm:f32[] cn:f32[]. let
          co:bool[] = convert_element_type[new_dtype=bool weak_type=False] cl
          cp:f32[] = select_n co cn cm
        in (cp,) }
      name=_where
    ] bv cj 0.0
    cq:f32[] = add bn ck
    cr:bool[] = lt x 0.0
    cs:bool[] = gt x 1.0
    ct:bool[] = convert_element_type[new_dtype=bool weak_type=False] cr
    cu:bool[] = convert_element_type[new_dtype=bool weak_type=False] cs
    cv:bool[] = or ct cu
    cw:f32[] = xla_call[
      call_jaxpr={ lambda ; cx:bool[] cy:f32[] cz:f32[]. let
          da:f32[] = select_n cx cz cy
        in (da,) }
      name=_where
    ] cv -inf cq
    db:f32[] = convert_element_type[new_dtype=float32 weak_type=False] cw
    dc:f32[] = reduce_sum[axes=()] db
    dd:f32[] = add 0.0 dc
    de:f32[] = add 0.0 dd
  in (g, b, v, b, v, b, v, dc, dd, de) }

This is how we've implemented `simulate` for this particular generative function language.^[In general, Gen doesn't require that we follow the same "code transformation" implementation for other generative function languages. GenJAX, however, is a bit special - because we restrict the user to remain within the JAX traceable subset of Python - any generative function interface implementation must also be JAX traceable. **This is a JAX requirement, not a Gen one**.]

## Generative function interface

There are a few more generative function interface methods worth discussing.

In this notebook, instead of carefully walking through the math which these interface methods compute, we'll defer that discussion to another notebook. Below, we give an informal discussion of what each of the interface methods computes, and roughly describe what algorithm families are supported by their usage.

### The generative function interface in GenJAX

GenJAX's generative functions define an interface which support compositional usage of generative functions within other generative functions. The interface functions here closely mirror [the interfaces defined in Gen.jl](https://www.gen.dev/docs/stable/ref/gfi/#Generative-function-interface-1).

In the following, we use the following abbreviations:

* **IS** - importance sampling
* **SMC** - sequential Monte Carlo
* **MCMC** - Markov chain Monte Carlo
* **VI** - variational inference

| Interface | Type | Inference algorithm support |
| --- | --- | --- |
| `simulate` | Generative | IS, SMC |
| `importance` | Generative | IS, SMC, VI |
| `update` | Generative and incremental | MCMC, SMC |
| `assess` | Generative and differentiable | MCMC, IS, SMC |
| `unzip` | Differentiable | Differentiable and involutive MCMC and SMC, VI |

This interface supports several methods - I've roughly described them and split them into the two categories **Generative** and **Differentiable** below:

#### Generative

* `simulate` - sample from normalized trace measure, and return the score.
* `importance` - given constraints for some addresses, sample from unnormalized trace measure and return an importance weight.
* `update` - given an existing trace, and a set of constraints and argument change values, update the trace to be consistent with the set of constraints under execution with the new arguments, and return an incremental importance weight.
* `assess` - given a complete choice map and arguments, return the normalized log probability.


#### Differentiable

* `assess` - same as above.
* `unzip` - given a set of fixed constraints, return two callables. The first callable `score` accepts constraints which fill in the complement of the fixed constraints and arguments, and returns the normalized log probability of all the constraints. The second callable `retval` accepts constraints and arguments, and returns the return value for the generative function call consistent with the constraints and given arguments.

**unzip** produces two functions which can be compositionally used with `jax.grad` to evaluate gradients used by both differentiable and involutive MCMC and SMC.

## More about generative functions

Here are a few more bits of information which should help you gain context with these objects.

### Distributions are generative functions

In GenJAX, distributions are generative functions.

In [6]:
key, tr = genjax.simulate(genjax.Normal)(key, (0.0, 1.0))
tr

DistributionTrace
├── gen_fn
│   └── _Normal
├── args
│   └── (0.0, 1.0)
├── value
│   └── ValueChoiceMap
│       └── value
│           └── f32[]
└── score
    └── f32[]

This should bring a sigh of relief! Ah, distributions are generative functions - *they can't be too exotic*.

Next to deterministic computations which trivially implement the interface, distributions are quite simple. They don't have internal compositional choice structure (like the function-like `BuiltinGenerativeFunction` language above).

Distributions themselves expose two interfaces:

* `logpdf` - exact density evaluation.
* `sample` - exact sampling.

We can use these two interfaces to implement all the generative function interfaces for distributions.

### Associated data types

* **Choice maps** are the tree-like recordings of random choices in a trace.
* **Selection** is an object which allows querying a trace/choice map - selecting certain choices.

In [7]:
@genjax.gen
def h(key, x):
    key, m1 = genjax.trace("m0", genjax.Bernoulli)(key, x)
    key, m2 = genjax.trace("m1", genjax.Bernoulli)(key, x)
    return (key, m1 + m2)


key, tr = genjax.simulate(h)(key, (0.3,))
tr

BuiltinTrace
├── gen_fn
│   └── BuiltinGenerativeFunction
│       └── source
│           └── <function h>
├── args
│   └── (0.3,)
├── retval
│   └── bool[]
├── choices
│   └── BuiltinChoiceMap
│       ├── m0
│       │   └── DistributionTrace
│       │       ├── gen_fn
│       │       │   └── _Bernoulli
│       │       ├── args
│       │       │   └── (0.3,)
│       │       ├── value
│       │       │   └── ValueChoiceMap
│       │       │       └── value
│       │       │           └── bool[]
│       │       └── score
│       │           └── f32[]
│       └── m1
│           └── DistributionTrace
│               ├── gen_fn
│               │   └── _Bernoulli
│               ├── args
│               │   └── (0.3,)
│               ├── value
│               │   └── ValueChoiceMap
│               │       └── value
│               │           └── bool[]
│               └── score
│                   └── f32[]
├── cache
│   └── BuiltinTrie
└── score
    └── f32[]

In [8]:
select = genjax.BuiltinSelection.new(["m1"])
selected, _ = select.filter(tr.get_choices())
selected

BuiltinChoiceMap
└── m1
    └── DistributionTrace
        ├── gen_fn
        │   └── _Bernoulli
        ├── args
        │   └── (0.3,)
        ├── value
        │   └── ValueChoiceMap
        │       └── value
        │           └── bool[]
        └── score
            └── f32[]

## Great ... now what can I do with them?

Now, we've informally seen the interfaces and datatypes associated with generative functions.

While studying the interfaces and the computational objects which satisfy them _in the abstract_ can be an entire PhD's worth of effort, we can do machine learning with them.

Let's consider a modeling problem where we wish to perform generalized regression with outliers between two variates, taking a family of polynomials as potential curves.

One such model for this data generating process is shown below.

In [9]:
# Two branches for a branching submodel.
@genjax.gen
def model_y(key, x, coefficients):
    key, y = trace("value", genjax.Normal)(key, 0.0, 1.0)
    return key, y


@genjax.gen
def outlier_model(key, x, coefficients):
    key, y = trace("value", genjax.Normal)(key, 0.0, 10.0)
    return key, y


# The branching submodel.
switch = genjax.SwitchCombinator([model_y, outlier_model])

# A mapped kernel function which calls the branching submodel.
@genjax.gen(genjax.MapCombinator, in_axes=(0, 0, None))
def kernel(key, x, coefficients):
    key, is_outlier = trace("outlier", genjax.Bernoulli)(key, 0.1)
    is_outlier = jnp.asarray(is_outlier, dtype=int)
    key, y = trace("y", switch)(key, is_outlier, x, coefficients)
    return key, y


@genjax.gen
def model(key, xs):
    key, coefficients = trace("alpha", genjax.MvNormal)(
        key, np.zeros(3), 2.0 * np.identity(3)
    )
    key, *sub_keys = jax.random.split(key, len(xs) + 1)
    sub_keys = jnp.array(sub_keys)
    _, ys = trace("ys", kernel)(sub_keys, xs, coefficients)
    return key, ys

There's a number of implementation patterns which you might pick up on by studying this model.

1. Generative functions explicitly pass a PRNG key in and out. This conforms to JAX's PRNG usage expectations.

2. To implement control flow, we use higher-order functions called combinators. These accept generative functions as input, and return generative functions as output.

3. Any JAX compatible code is allowed in the body of a generative function.

Courtesy of the interface, we get to design our `model` generative function in pieces. 

Now, let's examine a few sample traces from our model.

In [10]:
data = jnp.array([0.3, 1.0, 1.5, 2.0])
key, tr = jax.jit(model.simulate)(key, (data,))
tr

BuiltinTrace
├── gen_fn
│   └── BuiltinGenerativeFunction
│       └── source
│           └── <function model>
├── args
│   └── (f32[4],)
├── retval
│   └── f32[4]
├── choices
│   └── BuiltinChoiceMap
│       ├── alpha
│       │   └── DistributionTrace
│       │       ├── gen_fn
│       │       │   └── _MultivariateNormal
│       │       ├── args
│       │       │   └── (f32[3], f32[3,3])
│       │       ├── value
│       │       │   └── ValueChoiceMap
│       │       │       └── value
│       │       │           └── f32[3]
│       │       └── score
│       │           └── f32[]
│       └── ys
│           └── MapTrace
│               ├── gen_fn
│               │   └── MapCombinator
│               │       ├── kernel
│               │       │   └── BuiltinGenerativeFunction
│               │       │       └── source
│               │       │           └── <function kernel>
│               │       └── in_axes
│               │           └── (0, 0, None)
│               ├── indices
│               │   └── i32[4]
│               ├── inner
│               │   └── BuiltinTrace
│               │       ├── gen_fn
│               │       │   └── BuiltinGenerativeFunction
│               │       │       └── source
│               │       │           └── <function kernel>
│               │       ├── args
│               │       │   └── (f32[4], f32[4,3])
│               │       ├── retval
│               │       │   └── f32[4]
│               │       ├── choices
│               │       │   └── BuiltinChoiceMap
│               │       │       ├── outlier
│               │       │       │   └── DistributionTrace
│               │       │       │       ├── gen_fn
│               │       │       │       │   └── _Bernoulli
│               │       │       │       ├── args
│               │       │       │       │   └── (f32[4],)
│               │       │       │       ├── value
│               │       │       │       │   └── ValueChoiceMap
│               │       │       │       │       └── value
│               │       │       │       │           └── bool[4]
│               │       │       │       └── score
│               │       │       │           └── f32[4]
│               │       │       └── y
│               │       │           └── SwitchTrace
│               │       │               ├── gen_fn
│               │       │               │   └── SwitchCombinator
│               │       │               │       └── branches
│               │       │               │           └── [
│               │       │               │                 BuiltinGenerativeFunction(source = <function model_y>),
│               │       │               │                 BuiltinGenerativeFunction(source = <function 
│               │       │               │               outlier_model>)
│               │       │               │               ]
│               │       │               ├── chm
│               │       │               │   └── IndexedChoiceMap
│               │       │               │       ├── BuiltinTrace
│               │       │               │       │   ├── gen_fn
│               │       │               │       │   │   └── BuiltinGenerativeFunction
│               │       │               │       │   │       └── source
│               │       │               │       │   │           └── <function model_y>
│               │       │               │       │   ├── args
│               │       │               │       │   │   └── (f32[4], f32[4,3])
│               │       │               │       │   ├── retval
│               │       │               │       │   │   └── f32[4]
│               │       │               │       │   ├── choices
│               │       │               │       │   │   └── BuiltinChoiceMap
│               │       │               │       │   │       └── value
│               │       │               │       │   │           └── DistributionTrace
│               │       │               │       │   │               ├── gen_fn
│            

## Your first inference program